In [1]:
import time
import numpy as np
import pandas as pd

from collections import Counter
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

train = pd.DataFrame(pd.read_csv('train.csv'))
test = pd.DataFrame(pd.read_csv('test.csv'))
#train = train[:10000]

codes = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
         'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']

def one_hot_encode(seq):
    o = list(set(codes) - set(seq))
    s = pd.DataFrame(list(seq))    
    x = pd.DataFrame(np.zeros((len(seq),len(o)),dtype=int),columns=o)    
    a = s[0].str.get_dummies(sep=',')
    a = a.join(x)
    a = a.sort_index(axis=1)
    e = a.values.flatten()
    return e

t1 = time.clock()
X = train.Sequence.apply(lambda x: pd.Series(one_hot_encode(x)),1)
y = train.Active
t2 = time.clock()
print("Elapsed time: %.3f" %(t2-t1))

train_split = 0.9
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=train_split)
print(F'train/valid size: {len(X_train)}/{len(X_valid)}')

Elapsed time: 684.400
train/valid size: 100800/11200


In [3]:
t1 = time.clock()
clf = MLPClassifier(hidden_layer_sizes=(200,), max_iter=400)
clf.fit(X_train, y_train)
t2 = time.clock()
print("Elapsed time: %.3f" %(t2-t1))

print('f1 score, train:', f1_score(y_train,clf.predict(X_train)))
print('f1 score, valid:', f1_score(y_valid,clf.predict(X_valid)))
print(clf)

Elapsed time: 803.017
f1 score, train: 0.9984131182226924
f1 score, valid: 0.8955582232893158
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(200,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=400,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)


In [4]:
X_test = test.Sequence.apply(lambda x: pd.Series(one_hot_encode(x)),1)
Y_test = clf.predict(X_test)
df = pd.Series(Y_test)
df.to_csv('labels.csv', index=False, header=False)